In [45]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm 
import datetime as dt
import pytz
import glob

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
kakao_api_key = os.getenv('KAKAO_REST_API_KEY')

In [46]:
def get_seoul_time():
    seoul_tz = pytz.timezone('Asia/Seoul')
    seoul_time = dt.datetime.now(seoul_tz)
    return seoul_time
current_date = get_seoul_time()
current_str = current_date.strftime('%y%m%d')
current_str

'241107'

## 든든전세 페이지 크롤링

In [47]:
def parse_oneline(txt):
    output = [i for i in txt.split('\n')]
    return output

def parse_onepage(page_num):
    time.sleep(0.5)
    html=requests.get(f"https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={page_num}", verify=False)
    tb_TF = False
    try_cnt=0
    while not tb_TF:
        if try_cnt>10:
            raise
        time.sleep(1) 
        bs_test = BeautifulSoup(html.text, 'html.parser')
        if bs_test.find('table'):
            tb_TF = True 
        else:
            tb_TF = False
        try_cnt+=1
        
    tables = bs_test.find('table')
    thead = tables.find('thead')
    tab_cols = parse_oneline(thead.get_text().strip())
    tbody = tables.find('tbody')
    output_tab = pd.DataFrame([parse_oneline(i.strip()) for i in tbody.get_text().strip().split('\n\n')], columns=tab_cols)
    output_tab = output_tab.assign(href_id = [re.search('no=\d{10}', i['href']).group() for i in tbody.find_all(href=True)])
    return output_tab

In [49]:
sample = parse_onepage(1) 
service_date = sample.loc[0, '공고일자']

In [50]:
list_num = 50
datas = [parse_onepage(n) for n in tqdm(range(1,list_num+1))]

  0%|          | 0/13 [00:00<?, ?it/s]

In [51]:
final_data = pd.concat(datas).reset_index(drop=True).assign(address = lambda df: df['주소'].apply(lambda x: x.split('  ')[0] if ',' not in x else  x.split('  ')[0].split(',')[0])
                                               # .apply(lambda x: ' '.join([i for i in x.split(' ') if i !='']))
                                              )
final_data.to_csv(f'data/crawling{service_date}_{current_str}.csv', index = False, encoding = 'utf-8-sig')

In [52]:
final_data=pd.read_csv(f'data/crawling{service_date}_{current_str}.csv')

## 카카오 api 설정
### 주소 좌표 변환(위경도)

In [ ]:
import requests
import json
class kakaomap_rest_api:
    def __init__(self, api_token):
        self.api_token = api_token
    
    def convert_address_to_coordinates(self, address):
        """
        입력받은 주소를 WGS84 좌표계 좌표로 변환(카카오맵api)
        """
    
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        
        header = {'Authorization': 'KakaoAK ' + self.api_token}
     
        r = requests.get(url, headers=header)
        
        if (r.status_code == 200) and len(r.json()["documents"])>0:
            lng = float(r.json()["documents"][0]["address"]['x'])
            lat = float(r.json()["documents"][0]["address"]['y'])
        else:
            return None
        return lat, lng

    
    def search_by_category(self, category_group_code,  x, y, radius):
        """
        카테고리로 장소를 검색하는 함수
        
        Args:
            api_key (str): 카카오 개발자 REST API 키
            category_group_code (str): 카테고리 그룹 코드
            x(float): 경도(longitude)
            y(float): 위도(latitude)
            radius (int): 검색 반경 (미터 단위)
            
        
        Returns:
            dict: 검색 결과
        """
        url = "https://dapi.kakao.com/v2/local/search/category.json"
        
        headers = {
            "Authorization": f"KakaoAK {self.api_token}",
            "Content-Type": "application/json"
        }
        
        params = {
            "category_group_code": category_group_code,
            "radius": radius,
            "x":f"{x}",
            "y":f"{y}",
            "sort":"distance",
            "size":"5"
        }
        
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            print(f"API 요청 중 오류 발생: {e}")
            return None

    def calculate_transit_time(self, origin_y, origin_x, dest_y, dest_x):
        time.sleep(0.2)
        url = "https://apis-navi.kakaomobility.com/v1/directions"
        headers = {"Authorization": f"KakaoAK {self.api_token}"}
        params = {
            "origin": f"{origin_y},{origin_x}",
            "destination": f"{dest_y},{dest_x}",
            "priority": "RECOMMEND",
            "car_fuel": "GASOLINE",
            "car_hipass": True,
            "alternatives": False,
            "road_details": False,
            "roadevent":2
        }
        
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                result = response.json()
                return result['routes'][0]['summary']['duration'] / 60, result['routes'][0]['summary']['distance']
            return None
        except:
            return None
            
kakaomap = kakaomap_rest_api(kakao_api_key)

In [54]:
coordinates = [kakaomap.convert_address_to_coordinates(i) for i in tqdm(final_data.address.values)]

  0%|          | 0/122 [00:00<?, ?it/s]

In [55]:
final_data0 = pd.concat([final_data, pd.DataFrame(coordinates, columns = ['x','y'])], axis=1)

In [56]:
final_data0

,번호,공고일자,청약 접수기간,시도,시군구,주소,주택유형,전용면적(m2),임대보증금액,신청자수,href_id,address,x,y
0,122,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강동구,"서울 강동구 천호동 562, 562-1 스카이캐슬라 8층 802호",다세대주택,15.21,"152,100,000",668,no=2023040345,서울 강동구 천호동 562,37.539597,127.130622
1,121,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강서구,서울 강서구 등촌동 637-19 라빌라스 101동 4층 402호,오피스텔(주거용),29.99,"210,600,000",161,no=2022367225,서울 강서구 등촌동 637-19 라빌라스 101동 4층 402호,37.556256,126.859242
2,120,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강서구,"서울 강서구 등촌동 643-16, 643-17 라테라스 4층 404호",다세대주택,33.51,"251,100,000",133,no=2022362498,서울 강서구 등촌동 643-16,37.555297,126.859798
3,119,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강서구,서울 강서구 화곡동 105-207 바로크빌 4층 402호,다세대주택,29.90,"142,200,000",62,no=2022348197,서울 강서구 화곡동 105-207 바로크빌 4층 402호,37.539887,126.844564
4,118,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강서구,"서울 강서구 화곡동 1111, 1111-1 에스제이라벨라 10층 1002호",오피스텔(주거용),20.57,"143,100,000",196,no=2023198064,서울 강서구 화곡동 1111,37.554948,126.852357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,5,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,경기도,경기 부천시 원미구,경기 부천시 원미구 원미동 169-1 수팰리스 B동 4층 403호,다세대주택,34.92,"144,000,000",6,no=2022363045,경기 부천시 원미구 원미동 169-1 수팰리스 B동 4층 403호,37.494132,126.792039
118,4,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,경기도,경기 부천시 원미구,"경기 부천시 원미구 원미동 200-2, 200-4 송원리치빌 4층 402호",다세대주택,58.20,"154,800,000",11,no=2023204224,경기 부천시 원미구 원미동 200-2,37.488685,126.789119
119,3,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,경기도,경기 부천시 원미구,"경기 부천시 원미구 원미동 201-14, 201-17, 201-18 수팰리스 2층 ...",다세대주택,67.15,"190,800,000",15,no=2022388671,경기 부천시 원미구 원미동 201-14,37.488593,126.789558
120,2,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,경기도,경기 부천시 원미구,경기 부천시 원미구 원미동 42-1 해냄스토리주건축물 1동 10층 1002호,오피스텔(주거용),57.87,"226,800,000",36,no=2023205918,경기 부천시 원미구 원미동 42-1 해냄스토리주건축물 1동 10층 1002호,37.498914,126.789699


In [ ]:
final_data0.to_csv(f'data/crawling{service_date}_{current_str}_addcoord.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
final_data0 = pd.read_csv("data/crawling20250326_250331_addcoord.csv")

## 해당 매물 구조도 이미지 가져오기

In [57]:
def get_img_link(href_id, dt='20241007', max_retries=3):
    page_url = f"https://www.khug.or.kr/jeonse/web/s07/s070103.jsp?dt={dt}&{href_id}"
    
    for attempt in range(max_retries):
        try:
            # SSL 검증 비활성화 및 타임아웃 설정
            response = requests.get(page_url, verify=False, timeout=10)
            response.raise_for_status()
            
            bs_test = BeautifulSoup(response.content, 'html.parser')
            img_src = bs_test.find(id='imgSor0')
            
            if img_src is not None:
                return img_src.get('src')
            
            # 이미지를 찾지 못한 경우 짧은 대기 후 재시도
            time.sleep(0.2)
            
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(0.2)
            else:
                print(f"Error fetching {href_id}: {e}")
                return None
    
    return None

# SSL 경고 메시지 숨기기
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 배치 처리 방식으로 이미지 링크 수집
def get_all_img_links_batch(href_ids, dt='20241007', batch_size=5):
    results = []
    for i in tqdm(range(0, len(href_ids), batch_size)):
        batch = href_ids[i:i + batch_size]
        for href_id in batch:
            img_link = get_img_link(href_id, dt)
            results.append(img_link)
        # 배치 처리 후 잠시 대기
        time.sleep(1.5)
    return results



In [58]:
# 실행
imgs = get_all_img_links_batch(final_data0.href_id.values, dt=service_date, batch_size=5)

  0%|          | 0/122 [00:00<?, ?it/s]

In [59]:
final_data = final_data0.assign(img = imgs)

In [60]:
final_data.to_csv(f'data/data{service_date}_{current_str}.csv', index = False, encoding = 'utf-8-sig')

In [61]:
final_data.head(1)

,번호,공고일자,청약 접수기간,시도,시군구,주소,주택유형,전용면적(m2),임대보증금액,신청자수,href_id,address,x,y,img
0,122,20241031,2024.10.31. 10:00 ~ 2024.11.14. 17:00,서울특별시,서울 강동구,"서울 강동구 천호동 562, 562-1 스카이캐슬라 8층 802호",다세대주택,15.21,"152,100,000",668,no=2023040345,서울 강동구 천호동 562,37.539597,127.130622,http://www.khug.or.kr/updata/khgc/khgccms/cms/...


In [62]:
processing_data = pd.read_csv(f'data/data{service_date}_{current_str}.csv')

## 지하철 역 거리 계산

In [63]:
import geopandas as gpd
from shapely.geometry import Point
import shapely

In [64]:
processing_final = processing_data.assign(
    deposit = lambda df: df['임대보증금액'].apply(lambda x: int(x.replace(',', ''))/10000),
    m2 = lambda df: df['전용면적(m2)'],
    deposit_m2 = lambda df: df.deposit/df.m2)


In [65]:
processing_final.columns


In [66]:
sample_x, sample_y = processing_final.loc[0,['x', 'y']]

In [67]:
def find_near_subway_station(x, y, max_distance = 3000):
    time.sleep(0.2)
    results = kakaomap.search_by_category('SW8', y, x, 3000) # 위경도 바꿔어서 입력
    if len(results.get('documents'))!=0:
        near_result = results.get('documents')[0]
        return near_result.get('distance'), near_result.get('place_name')
    else:
        print('no result')
    

In [68]:
near_stations = [find_near_subway_station(row.x, row.y) for row in tqdm(processing_final.itertuples(), total = processing_final.shape[0])]

In [69]:
final = pd.concat([processing_final, pd.DataFrame(near_stations, columns = ["distanceM_near_station", "near_station"])], axis=1)

In [ ]:
final.to_csv(f'data/data{service_date}_{current_str}_addstation.csv', index = False, encoding = 'utf-8-sig')

In [70]:
final = pd.read_csv(f'data/data{service_date}_{current_str}_addstation.csv')

## 통근시간 계산

In [86]:
from dateutil import relativedelta
tomorrow = current_date+relativedelta.relativedelta(days=1)

In [118]:
comp_x, comp_y =  37.5058315272521, 127.040806473603


In [139]:
expected_times = [kakaomap.calculate_transit_time(i.y, i.x, comp_y, comp_x) for i in tqdm(final.itertuples(), total = final.shape[0])]

  0%|          | 0/122 [00:00<?, ?it/s]

In [144]:
final_csv = pd.concat([final, pd.DataFrame(expected_times, columns = ['expected_time', 'distance_comp'])], axis=1)
final_csv.to_csv(f'data/final{service_date}_{current_str}.csv', index = False, encoding='utf-8-sig')

## 신청자수 업데이트

In [ ]:
list_num = 50
datas = [parse_onepage(n) for n in tqdm(range(1,list_num+1))]

In [ ]:
final_data = pd.concat(datas).reset_index(drop=True).assign(address = lambda df: df['주소'].apply(lambda x: x.split('  ')[0] if ',' not in x else  x.split('  ')[0].split(',')[0])
                                               # .apply(lambda x: ' '.join([i for i in x.split(' ') if i !='']))
                                              )
final_data.to_csv(f'data/crawling{service_date}_{current_str}.csv', index = False, encoding = 'utf-8-sig')
csv_list = sorted(glob.glob(f"data/final{service_date}_*.csv"))
final_csv=pd.read_csv(csv_list[-1])

In [ ]:
final_csv0 = final_csv.drop(columns = '신청자수')
final_csv0['신청자수'] = final_data['신청자수']
final_csv0.to_csv(f'data/final{service_date}_{current_str}.csv', index = False, encoding='utf-8-sig')

## 이미지 다운로드 및 llm 기반 방구조도 분석

In [ ]:
from PIL import Image
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import io
import base64
import glob
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
class RoomDecription(BaseModel):
    num_room: int = Field(description="방의 수")
    num_balcony: int = Field(description="발코니 수")
    num_wc: int = Field(description="화장실 수")
    description: str = Field(description="방구조에 대한 설명")
parser = PydanticOutputParser(pydantic_object=RoomDecription)

def describe_img(img_url):
    response = requests.get(img_url)
    if response.status_code != 200:
        print(f"이미지 다운로드 실패: {response.status_code}")
    
    # 이미지를 base64로 인코딩
    image = Image.open(io.BytesIO(response.content))
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    
    # 다중 모달 메시지 구성
    message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": """
                다음 이미지는 방구조도 이미지입니다. 한글로 방의 구조를 설명해주세요.
                 - 빨간색 테두리가 있거나 붉은색으로 표시되어 있다면 해당 방만 설명해주세요.
                 - 방구조에 대한 설명은 불렛포인트를 사용해서 자세히 출력해주세요.
                 - 방이나 화장실의 갯수를 알려주세요.
                 - 이미지의 기반한 내용만 알려주세요.
                 - 다음 json 포맷에 맞게 출력해주세요.
                 ```json
                {{
                num_room:(방의 수),
                num_balcony:(발코니 수),
                num_wc:(화장실 수),
                description:(방구조 설명)
                }}
                 ```
                """
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{img_str}"},
            },
        ],
    )
    llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite')
    ans = llm.invoke([message])
    try:
        info = parser.parse(ans.content)
        return info
    except:
        print("JSON 파싱 실패")
        return None

In [ ]:
iterator_rows = final_csv.itertuples()
sample = next(iterator_rows)
sample

In [ ]:
ans = describe_img(sample.img)

### 이미지 다운로드 

In [ ]:
def download_image(img_url, save_path, filename, max_retries=3, timeout=30):
    for attempt in range(max_retries):
        try:
            # SSL 검증 비활성화 및 타임아웃 설정
            response = requests.get(img_url, verify=False, timeout=timeout)
            response.raise_for_status()
            
            os.makedirs(save_path, exist_ok=True)
            file_path = os.path.join(save_path, filename)
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            return True
            
        except requests.Timeout:
            if attempt < max_retries - 1:
                print(f"타임아웃 발생 ({filename}), {attempt + 1}/{max_retries} 재시도")
                time.sleep(2)  # 타임아웃 발생 시 더 긴 대기
            else:
                print(f"최대 타임아웃 재시도 횟수 초과 ({filename})")
                return False
                
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"다운로드 시도 {attempt + 1}/{max_retries} 실패: {e}")
                time.sleep(1)
            else:
                print(f"최대 재시도 횟수 초과 ({filename}): {e}")
                return False

def download_images_batch(df, batch_size=2, save_path='downloaded_images', max_retries=3, 
                         timeout=30, delay_between_batches=3):
    downloaded_paths = []
    
    # 이미지 다운로드
    for i in tqdm(range(0, df.shape[0], batch_size), desc="이미지 다운로드 중"):
        batch = df.loc[i:i + batch_size, :]
        
        for row in batch.itertuples():
            img_url = row.img
            if img_url:
                filename = f"{row.번호}.jpg"
                if download_image(img_url, save_path, filename, max_retries=max_retries, timeout=timeout):
                    downloaded_paths.append(os.path.join(save_path, filename))
                else:
                    print(f"이미지 다운로드 실패: {filename}")
        
        # 배치 처리 후 대기
        time.sleep(delay_between_batches)
        
        # # 진행 상황 저장 (선택사항)
        # progress = {
        #     'downloaded': downloaded_paths,
        #     'current_index': i + batch_size
        # }
        # with open('download_progress.json', 'w') as f:
        #     json.dump(progress, f)
    
    return downloaded_paths



In [ ]:
# 실행
downloaded_paths = download_images_batch(
    final_csv.loc[:30,:], 
    batch_size=3,  # 배치 사이즈
    max_retries=3,
    timeout=30,    # 타임아웃 시간
    delay_between_batches=5  # 배치 간 대기 시간
)